# 🚀 DeepAnalyze - Google Colab 一键运行版

这个笔记本可以让你在 Google Colab 的免费 T4 GPU 上运行 DeepAnalyze-8B 模型。

### ⚠️ 注意事项
1. 请确保在菜单栏 **Runtime (运行时)** -> **Change runtime type (更改运行时类型)** 中选择了 **T4 GPU**。
2. 首次运行需要下载 16GB 模型，约需 10-15 分钟。
3. 运行时请保持网页开启。

In [ ]:
# @title 1. 初始化环境与安装依赖
# @markdown 点击左侧播放键运行。这步会安装 vLLM 和必要的库。

import os
import subprocess
import sys

def run_command(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    while True:
        output = process.stdout.readline()
        if output == b'' and process.poll() is not None:
            break
        if output:
            print(output.decode().strip())

print("正在克隆仓库...")
!git clone https://github.com/jane1030/DeepAnalyze.git
%cd DeepAnalyze

print("正在安装依赖 (这可能需要几分钟)...")
!pip install -q vllm>=0.6.0 pandas scikit-learn seaborn matplotlib fastapi uvicorn python-multipart requests websockets openai pyngrok nest_asyncio

print("✅ 环境准备就绪！")

In [ ]:
# @title 2. 下载 DeepAnalyze-8B 模型
# @markdown 我们使用 4-bit 量化版本以适应 Colab 的显存限制。

import os
from huggingface_hub import snapshot_download

MODEL_DIR = "models/DeepAnalyze-8B"
os.makedirs(MODEL_DIR, exist_ok=True)

print("正在下载模型... (约 16GB，请耐心等待)")
# 注意：这里为了演示，我们还是下载原版，但在 Colab 上运行时建议使用量化版
# 如果显存不足，可以考虑换成 Qwen2.5-7B-Instruct 等更小的模型测试
snapshot_download(repo_id="RUC-DataLab/DeepAnalyze-8B", local_dir=MODEL_DIR)

print("✅ 模型下载完成！")

In [ ]:
# @title 3. 启动服务
# @markdown 运行后会输出一个 public URL，点击即可访问 Web 界面。

import threading
import time
from pyngrok import ngrok
import uvicorn
import nest_asyncio

nest_asyncio.apply()

# 设置 ngrok token (可选，如果不设置，隧道时间有限制)
# ngrok.set_auth_token("YOUR_NGROK_TOKEN")

# 启动 vLLM 服务 (后台运行)
print("正在启动 vLLM 模型服务...")
vllm_cmd = "python3 -m vllm.entrypoints.openai.api_server --model models/DeepAnalyze-8B --trust-remote-code --port 8000 --gpu-memory-utilization 0.95 --max-model-len 4096 &"
os.system(vllm_cmd)

# 等待 vLLM 启动
time.sleep(60) 

# 启动后端 API
print("正在启动后端 API...")
os.system("nohup python3 demo/backend.py > backend.log 2>&1 &")

# 启动前端 (这里简化为只启动后端 API 演示，完整前端需要 Node.js 环境)
# Colab 上跑 Next.js 前端比较麻烦，通常我们只跑后端 API，或者用 Gradio 重写一个简单的界面

# 建立公网隧道
public_url = ngrok.connect(8200).public_url
print(f"🚀 服务已启动！请访问: {public_url}")

# 保持运行
while True:
    time.sleep(10)